### import libraries and load data

In [1]:
import numpy as np
import pandas as pd 

In [2]:
cart = pd.read_csv("DataAnalyst_Ecom_data_addsToCart.csv")
session = pd.read_csv("DataAnalyst_Ecom_data_sessionCounts.csv")

### Sheet 1

In [3]:
month_device = session.groupby(["dim_date", "dim_deviceCategory"]).sum()
month_device['ECR'] = month_device['transactions'] / month_device['sessions'] 
month_device

sessions  transactions  QTY       ECR
dim_date dim_deviceCategory                                       
1/1/13   desktop                 4773           182  304  0.038131
         mobile                  4070            22   67  0.005405
         tablet                  1892            22   29  0.011628
1/10/13  desktop                 9277           282  612  0.030398
         mobile                 26404           296  535  0.011210
...                               ...           ...  ...       ...
9/8/12   mobile                  5455            62   92  0.011366
         tablet                  1861            24   21  0.012896
9/9/12   desktop                 8192           164  333  0.020020
         mobile                   224             3   11  0.013393
         tablet                  2913            42   44  0.014418

[1095 rows x 4 columns]

### Sheet 2

Getting the 2 most recent months of data and get the total of each field during the period (for session)

In [4]:
cart2month = cart[cart['dim_year'] == cart['dim_year'].max()]
cart2month = cart2month[cart2month['dim_month'].isin(pd.Series(pd.unique(cart2month['dim_month'])).nlargest(2))]
cart2month

,dim_year,dim_month,addsToCart
10,2013,5,136720
11,2013,6,107970


In [5]:
session['dim_year'] = pd.to_datetime(session['dim_date']).dt.year
session['dim_month'] = pd.to_datetime(session['dim_date']).dt.month
session2month = session[session['dim_year'] == session['dim_year'].max()]
session2month = session2month[session2month['dim_month'].isin(pd.Series(pd.unique(session2month['dim_month'])).nlargest(2))]
session2month = session2month.groupby(["dim_year", "dim_month"]).sum()
session2month

sessions  transactions    QTY
dim_year dim_month                               
2013     5           1164639         28389  51629
         6           1388834         34538  61891

Merge the cart and session table

In [6]:
MoM = session2month.merge(cart2month, on=["dim_month", "dim_year"])
MoM

,dim_month,dim_year,sessions,transactions,QTY,addsToCart
0,5,2013,1164639,28389,51629,136720
1,6,2013,1388834,34538,61891,107970


Calculate the absolute and relative difference

In [7]:
for i in ['sessions', 'transactions', 'QTY', 'addsToCart']:
    MoM[i+"_abs_diff"] = MoM[i].diff()
    MoM[i+"_rel_diff"] = MoM[i].pct_change()
MoM

,dim_month,dim_year,sessions,transactions,QTY,addsToCart,sessions_abs_diff,sessions_rel_diff,transactions_abs_diff,transactions_rel_diff,QTY_abs_diff,QTY_rel_diff,addsToCart_abs_diff,addsToCart_rel_diff
0,5,2013,1164639,28389,51629,136720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,2013,1388834,34538,61891,107970,224195.0,0.192502,6149.0,0.216598,10262.0,0.198764,-28750.0,-0.210284


### To excel

In [8]:
with pd.ExcelWriter('output.xlsx') as writer:  
    month_device.to_excel(writer, sheet_name='month_device')
    MoM.to_excel(writer, sheet_name='MoM')

#### End of Coding Exercise

#### Data For Visualization / Presentation

In [10]:
merged = session.groupby(["dim_year", "dim_month"]).sum().merge(cart, on=["dim_month", "dim_year"])
for i in ['sessions', 'transactions', 'QTY', 'addsToCart']:
    merged[i+"_abs_diff"] = merged[i].diff()
    merged[i+"_rel_diff"] = merged[i].pct_change()
merged.to_csv("session_cart.csv", index=False)